In [1]:
import pickle
import random
import numpy as np

random.seed(42)

In [2]:
# X = {file_name: (frames, joints (xyz) * 25)}
with open('X.pkl', 'rb') as f:
    X_raw = pickle.load(f)

In [3]:
train_x = {}
test_x = {}

# [{actor1: [frame], actor2: [frame]}]
train = []
test = [] 
same_samples_per_actor = 5000
diff_samples_per_actor = 5000

for file in X_raw:
    actor = int(file[9:12])
    action = int(file[17:20])
    
    is_train = action <= 60
    if is_train:
        if actor not in train_x:
            train_x[actor] = []
        if len(X_raw[file]) == 0:
            continue
        train_x[actor].append(X_raw[file])
    else:
        if actor not in test_x:
            test_x[actor] = []
        if len(X_raw[file]) == 0:
            continue
        test_x[actor].append(X_raw[file])

for actor in train_x:
    for i in range(same_samples_per_actor):
        # Randomly select 2 frames from the same actor
        video1 = random.choice(train_x[actor])
        video2 = random.choice(train_x[actor])

        # Pad or clip video to 50 frames
        if len(video1) < 50: video1 = np.pad(video1, ((0, 50 - len(video1)), (0, 0)), 'constant')
        elif len(video1) > 50: video1 = video1[:50]
        if len(video2) < 50: video2 = np.pad(video2, ((0, 50 - len(video2)), (0, 0)), 'constant')
        elif len(video2) > 50: video2 = video2[:50]
        
        # Flatten Video
        video1 = video1.flatten()
        video2 = video2.flatten()

        # Add to train
        train.append({actor: np.array([video1, video2]), 'is_same': True})

    for i in range(diff_samples_per_actor):
        # Randomly select 2 frames from different actors
        while True: # Make sure the 2 frames are not from the same actor
            random_actor = random.choice(list(train_x.keys()))
            if random_actor != actor:
                video1 = random.choice(train_x[actor])
                video2 = random.choice(train_x[random_actor])

                # Pad or clip video to 50 frames
                if len(video1) < 50: video1 = np.pad(video1, ((0, 50 - len(video1)), (0, 0)), 'constant')
                elif len(video1) > 50: video1 = video1[:50]
                if len(video2) < 50: video2 = np.pad(video2, ((0, 50 - len(video2)), (0, 0)), 'constant')
                elif len(video2) > 50: video2 = video2[:50]

                # Flatten Video
                video1 = video1.flatten()
                video2 = video2.flatten()

                # Add to train
                train.append({actor: video1, random_actor: video2, 'is_same': False})
                break

for actor in test_x:
    for i in range(same_samples_per_actor):
        # Randomly select 2 frames from the same actor
        video1 = random.choice(test_x[actor])
        video2 = random.choice(test_x[actor])

        # Pad or clip video to 50 frames
        if len(video1) < 50: video1 = np.pad(video1, ((0, 50 - len(video1)), (0, 0)), 'constant')
        elif len(video1) > 50: video1 = video1[:50]
        if len(video2) < 50: video2 = np.pad(video2, ((0, 50 - len(video2)), (0, 0)), 'constant')
        elif len(video2) > 50: video2 = video2[:50]

        # Flatten Video
        video1 = video1.flatten()
        video2 = video2.flatten()

        # Add to test
        test.append({actor: np.array([video1, video2]), 'is_same': True})

    for i in range(diff_samples_per_actor):
        # Randomly select 2 frames from different actors
        while True: # Make sure the 2 frames are not from the same actor
            random_actor = random.choice(list(test_x.keys()))
            if random_actor != actor:
                video1 = random.choice(test_x[actor])
                video2 = random.choice(test_x[random_actor])
                
                # Pad or clip video to 50 frames
                if len(video1) < 50: video1 = np.pad(video1, ((0, 50 - len(video1)), (0, 0)), 'constant')
                elif len(video1) > 50: video1 = video1[:50]
                if len(video2) < 50: video2 = np.pad(video2, ((0, 50 - len(video2)), (0, 0)), 'constant')
                elif len(video2) > 50: video2 = video2[:50]
                
                # Flatten Video
                video1 = video1.flatten()
                video2 = video2.flatten()

                # Add to test
                test.append({actor: video1, random_actor: video2, 'is_same': False})
                break

train = np.array(train)
test = np.array(test)

KeyboardInterrupt: 

In [ ]:
# Train
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, confusion_matrix, precision_score, recall_score, f1_score

X_train = []
y_train = []

for sample in train:
    if sample['is_same']:
        for actor in sample:
            if actor != 'is_same':
                X_train.append(np.concatenate([sample[actor][0], sample[actor][1]]))
        y_train.append(1)
    else:
        temp = []
        for actor in sample:
            if actor != 'is_same':
                temp.append(sample[actor])
        X_train.append(np.concatenate(temp))
        y_train.append(0)

# X_train = np.array(X_train)
# y_train = np.array(y_train)

# I'll do a grid search later
clf = RandomForestClassifier(n_estimators=100, max_depth=30, random_state=42)
# clf.fit(X_train, y_train)

# Test
X_test = []
y_test = []

for sample in test:
    if sample['is_same']:
        for actor in sample:
            if actor != 'is_same':
                X_test.append(np.concatenate([sample[actor][0], sample[actor][1]]))
        y_test.append(1)
    else:
        temp = []
        for actor in sample:
            if actor != 'is_same':
                temp.append(sample[actor])
        X_test.append(np.concatenate(temp))
        y_test.append(0)

X_test = np.array(X_test)
y_test = np.array(y_test)

# y_pred = clf.predict(X_test)

# print('Accuracy: ', accuracy_score(y_test, y_pred))
# print('Precision: ', precision_score(y_test, y_pred))
# print('Recall: ', recall_score(y_test, y_pred))
# print('F1 Score: ', f1_score(y_test, y_pred))
# print('Confusion Matrix:\n', confusion_matrix(y_test, y_pred))

In [ ]:
from sklearn.neural_network import MLPClassifier

clf = MLPClassifier(hidden_layer_sizes=(1000, 100, 100), max_iter=1000, random_state=42)
clf.fit(X_train, y_train)

y_pred = clf.predict(X_test)

print('Accuracy: ', accuracy_score(y_test, y_pred))
print('Precision: ', precision_score(y_test, y_pred))
print('Recall: ', recall_score(y_test, y_pred))
print('F1 Score: ', f1_score(y_test, y_pred))
print('Confusion Matrix:\n', confusion_matrix(y_test, y_pred))

Accuracy:  0.6998275362318841
Precision:  0.7058582890654627
Recall:  0.6851797101449275
F1 Score:  0.6953653001281084
Confusion Matrix:
 [[246494  98506]
 [108613 236387]]


In [ ]:
from sklearn.linear_model import LogisticRegression

clf_lr = LogisticRegression(random_state=42)
clf_lr.fit(X_train, y_train)

y_pred = clf_lr.predict(X_test)

print('Accuracy: ', accuracy_score(y_test, y_pred))
print('Precision: ', precision_score(y_test, y_pred))
print('Recall: ', recall_score(y_test, y_pred))
print('F1 Score: ', f1_score(y_test, y_pred))
print('Confusion Matrix:\n', confusion_matrix(y_test, y_pred))

c:\Users\Carrt\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\linear_model\_logistic.py:444: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


Accuracy:  0.499895652173913
Precision:  0.499901655466317
Recall:  0.5304173913043478
F1 Score:  0.5147076196101594
Confusion Matrix:
 [[161934 183066]
 [162006 182994]]


In [ ]:
with open('mlp_classifier.pkl', 'wb') as f:
    pickle.dump(clf, f)